# Day 09. Exercise 02
# Metrics

## 0. Imports

In [177]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix
import joblib

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [178]:
df = pd.read_csv("../data/dayofweek-not-scaled.csv")

In [179]:
X = df.drop(['dayofweek'], axis=1)
y = df['dayofweek']

In [180]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y,
                                                    random_state=21)

## 2. SVM

1. Use the best parameters from the previous exercise and train the model of SVM.
2. You need to calculate `accuracy`, `precision`, `recall`, `ROC AUC`.

 - `precision` and `recall` should be calculated for each class (use `average='weighted'`)
 - `ROC AUC` should be calculated for each class against any other class (all possible pairwise combinations) and then weighted average should be applied for the final metric
 - the code in the cell should display the result as below:

```
accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97878
```

In [181]:
svc = SVC(C=10, class_weight=None, gamma='auto', kernel='rbf', probability=True)
svc.fit(X_train, y_train)

SVC(C=10, gamma='auto', probability=True)

In [182]:
y_pred = svc.predict(X_test)
print(f"accuracy is {accuracy_score(y_test, y_pred):.5f}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted'):.5f}")
y_score = svc.predict_proba(X_test)
print(f"roc_auc is {roc_auc_score(y_test, y_score, multi_class='ovo', average='weighted'):.5f}")

accuracy is 0.88757
precision is 0.89267
recall is 0.88757
roc_auc is 0.97868


## 3. Decision tree

1. The same task for decision tree

In [183]:
dt = DecisionTreeClassifier(max_depth=17, class_weight=None, criterion='gini', random_state=21)
dt.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=17, random_state=21)

In [184]:
y_pred = dt.predict(X_test)
print(f"accuracy is {accuracy_score(y_test, y_pred):.5f}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted'):.5f}")
y_score = dt.predict_proba(X_test)
print(f"roc_auc is {roc_auc_score(y_test, y_score, multi_class='ovo', average='weighted'):.5f}")

accuracy is 0.84615
precision is 0.85148
recall is 0.84615
roc_auc is 0.92659


## 4. Random forest

1. The same task for random forest.

In [185]:
rfc = RandomForestClassifier(class_weight='balanced', criterion='entropy', max_depth=26, n_estimators=100, random_state=21)
rfc.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=26, random_state=21)

In [186]:
y_pred = rfc.predict(X_test)
print(f"accuracy is {accuracy_score(y_test, y_pred):.5f}")
print(f"precision is {precision_score(y_test, y_pred, average='weighted'):.5f}")
print(f"recall is {recall_score(y_test, y_pred, average='weighted'):.5f}")
y_score = rfc.predict_proba(X_test)
print(f"roc_auc is {roc_auc_score(y_test, y_score, multi_class='ovo', average='weighted'):.5f}")

accuracy is 0.93195
precision is 0.93423
recall is 0.93195
roc_auc is 0.98862


## 5. Predictions

1. Choose the best model.
2. Analyze: for which `weekday` your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which `labname` and for which `users`.
3. Save the model.

In [187]:
rfc = RandomForestClassifier(class_weight='balanced', criterion='entropy', max_depth=26, n_estimators=100, random_state=21)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [188]:
df_forecast = pd.DataFrame({"predict":y_pred}, index= y_test.index)
df_fit = pd.DataFrame({"predict":rfc.predict(X_train)}, index= y_train.index)

df['forecast'] = pd.concat([df_fit, df_forecast])
df['Error'] = (df["forecast"]!=df.dayofweek)*1
error_analysis = df.groupby(['dayofweek']).agg(
    total_samples=('Error', 'size'),
    total_errors=('Error', 'sum')
).reset_index()
error_analysis['perc_error'] = (error_analysis.total_errors/error_analysis.total_samples).round(2)
error_analysis.sort_values(by = 'perc_error', ascending=False)
conf_matrix = confusion_matrix(df['dayofweek'], df['forecast'])
print(conf_matrix)

[[130   1   0   0   0   1   4]
 [  1 271   0   0   0   1   1]
 [  0   0 147   2   0   0   0]
 [  1   0   0 394   0   0   1]
 [  0   0   0   0 101   3   0]
 [  0   0   0   2   0 266   3]
 [  0   0   0   0   0   2 354]]


In [189]:
((df[df.Error == 1].iloc[:,3:31].sum()/df.iloc[:,3:31].sum()).round(2)).sort_values(ascending=False)

uid_user_6     0.17
uid_user_27    0.04
uid_user_16    0.03
uid_user_18    0.03
uid_user_30    0.03
uid_user_3     0.03
uid_user_31    0.03
uid_user_2     0.02
uid_user_24    0.02
uid_user_25    0.02
uid_user_19    0.02
uid_user_29    0.02
uid_user_10    0.01
uid_user_4     0.01
uid_user_14    0.01
uid_user_13    0.00
uid_user_1     0.00
uid_user_0     0.00
uid_user_11    0.00
uid_user_15    0.00
uid_user_17    0.00
uid_user_12    0.00
uid_user_26    0.00
uid_user_22    0.00
uid_user_20    0.00
uid_user_23    0.00
uid_user_21    0.00
uid_user_28    0.00
dtype: float64

In [190]:
((df[df.Error == 1].iloc[:,33:-2].sum()/df.iloc[:,33:-2].sum()).round(2)).sort_values(ascending=False)

labname_lab03       1.00
labname_laba04      0.03
labname_lab05s      0.03
labname_laba06s     0.02
labname_laba06      0.02
labname_laba04s     0.02
labname_code_rvw    0.01
labname_project1    0.01
labname_lab02       0.00
labname_lab03s      0.00
labname_laba05      0.00
dtype: float64

In [191]:
joblib.dump(rfc, '../model/forest_model_ex02.joblib')

['../model/forest_model_ex02.joblib']

## 6. Function

1. Write a function that takes a list of different models and a corresponding list of parameters (dicts) and returns a dict that contains all the 4 metrics for each model.

In [192]:
def evaluate_models(models, params_list, X_train, y_train, X_test, y_test):
    results = {}

    for model, params in zip(models, params_list):
        parametr = {key: value for key, value in params.items() if key in model.get_params()}
        model.set_params(**parametr)

        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)
        
        accuracy = round(accuracy_score(y_test, y_pred), 5)
        precision = round(precision_score(y_test, y_pred, average='weighted'), 5)
        recall = round(recall_score(y_test, y_pred, average='weighted'), 5)
        roc_auc = round(roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class='ovr').item(), 5)

        results[model.__class__.__name__] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': roc_auc
        }

    return results

In [193]:
# Список параметров для каждой модели
models = [SVC(),
          DecisionTreeClassifier(),
          RandomForestClassifier()]

# Список параметров для каждой модели
params_list = [
    {'probability': True, 'random_state': 21, 'kernel': 'linear'},
    {'max_depth': 17, 'random_state':21},
    {'n_estimators': 100, 'max_depth': 28, 'random_state':21},
]

# Оценка моделей
results = evaluate_models(models, params_list, X_train, y_train, X_test, y_test)

for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print(f"Accuracy: {metrics['accuracy']}")
    print(f"Precision: {metrics['precision']}")
    print(f"Recall: {metrics['recall']}")
    print(f"F1-score: {metrics['f1']}")
    print()

Model: SVC
Accuracy: 0.71893
Precision: 0.72714
Recall: 0.71893
F1-score: 0.92115

Model: DecisionTreeClassifier
Accuracy: 0.84615
Precision: 0.85148
Recall: 0.84615
F1-score: 0.9287

Model: RandomForestClassifier
Accuracy: 0.93195
Precision: 0.93313
Recall: 0.93195
F1-score: 0.9913

